# Semantic Kernel

For dis code sample, you go use di [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework to create one basic agent.

Di goal for dis sample na to show you di steps wey we go later use for di other code samples when we dey implement di different agentic patterns.


## Import di Python Packages wey we need


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## How to Create Client

For dis example, we go use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) to take get access to di LLM.

Di `ai_model_id` na `gpt-4o-mini`. You fit try change di model to another one wey dey GitHub Models marketplace to see di different results.

To fit use di `Azure Inference SDK` wey dem dey use for di `base_url` for GitHub Models, we go use di `OpenAIChatCompletion` connector inside Semantic Kernel. E still get other [connectors wey dey available](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) wey you fit use Semantic Kernel for other model providers.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## How to Create di Agent

For here, we go create one Agent wey we go call `TravelAgent`.

For dis example, we dey use very simple instructions. You fit change dis instructions to see how di agent go take respond differently.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## How to Run di Agent

Now we fit run di Agent by defining one thread wey be type `ChatHistoryAgentThread`. Any system message wey dem need go dey inside di agent invoke_stream `messages` keyword argument.

After we don define all dis one, we go create `user_inputs` wey go be wetin di user dey send give di agent. For dis example, we don set di message to `Plan me a sunny vacation`.

You fit change dis message if you wan see how di agent go respond differently.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am accurate, abeg make you sabi say machine translation fit get mistake or no dey correct well. Di original dokyument wey dey for im native language na di one wey you go take as di correct source. For important information, e good make you use professional human translation. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
